In [1]:
import matplotlib.pylab as plt
%matplotlib inline
import numpy as np
from PIL import ImageTk
from numpy.linalg import svd
from os import listdir, getcwd
from sympy import Matrix
from os.path import isfile, join
from random import randint
from PIL import Image
import cv2
from tkinter import *
from sklearn.decomposition import TruncatedSVD
import ttkbootstrap as tb
from ttkbootstrap.constants import *
root=tb.Window(themename='vapor')
root.title('Home')
root.geometry("450x400")
my_style=tb.Style()
my_style.configure('info.TButton',font=('Helvetica',18))
l=tb.Label(text='Atleast one object must be moving in your Video..',bootstyle='default')
l.grid(row=0,column=0,padx=(150,0),pady=20)
def rec():
    import cv2
    vidcap = cv2.VideoCapture(0)
    global count
    count = 0
    while (True):
        success,image = vidcap.read()
        cv2.imwrite("C:/Users/SS/Desktop/webcam/Frame %d.jpg" % count, image)     # save frame as JPEG file in a folder named webcam      
        cv2.imshow('Output', image)
        count += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):# q pres karna to stop your webcam
            break
    vidcap.release()
    cv2.destroyAllWindows()
def show():
    root1=Toplevel()
    from PIL import Image,ImageTk  
    def get_image():
        global li
        li=[]
        i=22
        while(i<count):
            img1=ImageTk.PhotoImage(Image.open("C:\\Users\\SS\\Desktop\\webcam\\Frame %d.jpg" % i))
            li.append(img1)
            i=i+1
        global l
        l=Label(root1,image=li[0])
        l.grid(row=1,column=0,columnspan=3)
        
    get_image()
    status = Label(root1,text='Image 1'+' of '+ str(len(li)),bd=1,relief=SUNKEN,anchor=E)
    status.grid(row=3,column=0,columnspan=3,sticky=W+E,pady=5)
    B_extract=tb.Button(root1,text="Extract Background",bootstyle='primary',command=extract)
    B_extract.grid(row=7,column=0,pady=(50,20),padx=(250,10))
    from PIL import Image,ImageTk
    def Back(i):     
            global l
            global forward
            global back
            l.grid_forget()
            l=Label(root1,image=li[i-1])
            forward=Button(root1,text='>>',command=lambda : ford(i+1))
            back=Button(root1,text='<<',command=lambda : Back(i-1))
            l.grid(row=1,column=0,columnspan=3)
            back.grid(row=2,column=0,sticky=W)
            forward.grid(row=2,column=2)
            status = Label(root1,text='Image '+str(i)+' of '+ str(len(li)),bd=1,relief=SUNKEN,anchor=E)
            status.grid(row=3,column=0,columnspan=3,sticky=W+E,pady=5)
            if i==1:
                back=Button(root1,text='<<',state=DISABLED)
                back.grid(row=2,column=0,sticky=W)
        
    def ford(i):
            global l
            global forward
            global back
            l.grid_forget()
            l=Label(root1,image=li[i-1])
            forward=Button(root1,text='>>',command=lambda : ford(i+1))
            back=Button(root1,text='<<',command=lambda : Back(i-1))
            l.grid(row=1,column=0,columnspan=3)
            back.grid(row=2,column=0,sticky=W)
            forward.grid(row=2,column=2)
            status = Label(root1,text='Image '+str(i)+' of '+ str(len(li)),bd=1,relief=SUNKEN,anchor=E)
            status.grid(row=3,column=0,columnspan=3,sticky=W+E,pady=5)
            if i==len(li):
                forward=Button(root1,text='>>',state=DISABLED)
                forward.grid(row=2,column=2)

    back=Button(root1,text='<<',state=DISABLED)
    forward=Button(root1,text='>>',command=lambda : ford(2))
    exit=Button(root1,text='Exit',command=root.destroy)
    exit.grid(row=2,column=1,padx=(0,300))
    back.grid(row=2,column=0,sticky=W)
    forward.grid(row=2,column=2)

rec_button=tb.Button(text='Record',bootstyle='primary',command= rec)
rec_button.grid(row=1,column=0,pady=100,padx=(150,0))
bf=tb.Button(text='Show Frames',bootstyle='primary',command= show)
bf.grid(row=2,column=0,pady=100,padx=(150,0))

exit=tb.Button(text='Exit',bootstyle='danger',command=root.destroy)
exit.grid(row=6,column=0,pady=(50,20),padx=(150,0))
def extract():
    from PIL import Image,ImageTk
    img = Image.open("C:\\Users\\SS\\Desktop\\webcam\\Frame 0.jpg")
    I = np.array(img)
    I=I[:,:,0]
    global Length
    global width
    global X
    Length, Width = I.shape
    X = np.zeros((count,Length*Width))
    
    for i in range(count):
        file="C:\\Users\\SS\\Desktop\\webcam\\Frame %d.jpg" % i
        img = Image.open(file)
        I = np.array(img)
        I=I[:,:,0]
        X[i, :] = I.reshape(1, -1)
    U, s, VT = svd(X, full_matrices=False)
    S = np.diag(s)
    L = 1
    Xhat = U[:, :L]@S[0:L, 0:L]@VT[:L, :]
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
    from matplotlib.figure import Figure
    
    root = Tk()
    root_panel = tb.Frame(root)
    root_panel.pack(side="bottom", fill="both", expand="yes")

    btn_panel = Frame(root_panel, height=35)
    btn_panel.pack(side='top', fill="both", expand="yes")
    f = Figure()
    a = f.add_subplot(111)
    a.imshow(Xhat[0,:].reshape(Length,Width),cmap="gray")
    canvas = FigureCanvasTkAgg(f, master=root)
    canvas.draw()
    canvas.get_tk_widget().pack(side="top", fill="both", expand=1)
    canvas._tkcanvas.pack(side="top", fill="both", expand=1)




root.mainloop()
